In [60]:
#For dataframes
import pandas as pd
#For other stuff
import numpy as np

#Read output from metaboigniter
data=pd.read_csv("../99-output/TABLE_OUTPUT/output_quantification_Linked_data.tsv", sep='\t')

In [61]:
#View dataframe
#print(data.head(5))

In [62]:
#Remove excess columns for ease of processing, can use the IDs later to cross-reference compounds to get m/z and RT info
data_trunc=data.drop(columns=['charge','RT','mz','quality','feature_ids'])

In [63]:
#print(data_trunc.head(5))

In [79]:
#Transpose dataframe to be of form [sample,feature]
data_t=data_trunc.transpose()

In [65]:
#print(data_t.head(5))

In [80]:
#Let's do some data cleanup

#Replace column headers with the first row in the dataframe
data_t.columns=data_t.iloc[0]

In [70]:
#print(data_t.head(5))

In [81]:
#Delete the repeated row
data_t=data_t.drop(data_t.index[0])

In [72]:
#print(data_t.head(5))

In [82]:
#Make 'sample' the new index and separate the columns
data_t.index.name='sample'
data_t.reset_index(inplace=True)
#data_t=data_t.set_index('sample')

In [75]:
#print(data_t.head(5))

In [83]:
#Split samplename-rep.ext into samplename, rep.ext columns
data_t[['sample','rep']] = data_t['sample'].str.split('-',expand=True)

In [77]:
#print(data_t.head(5))

In [84]:
#Split rep.ext in to rep and ext columns
data_t[['rep','ext']]=data_t['rep'].str.split('.',expand=True)

In [85]:
#Delete 'ext' column
data_t=data_t.drop(columns='ext')

In [86]:
#print(data_t.head(5))

In [88]:
#print(data_t.head(5))

In [ ]:
#Need code that will look to see if every repeated measure in at least one sample has a non-zero entry, otherwise remove that column
#new_df = pd.DataFrame()
#df = pd.DataFrame()
#for col in data_t.columns:
#    for row in data_t.index:
#        df=data_t[row,col]
#        if not 0 in df:
#            new_df['col']=data_t['col']
#            break


#None of this seems to work, need a new approach

In [90]:
#Create new list for column headers to keep
keep=[]

# Goes column by column (peak by peak), defines a function to check for zeroes, keeps only peaks where at least one set of injections 
# has all non-zero entries, otherwise toss it
for col in data_t.columns:
    has_zero = data_t.groupby(data_t['sample'])[col].transform(lambda x: (x == 0).any())
    if not has_zero.all():
        keep.append(col)

In [91]:
#Check to see how many peaks we've kept
print(len(keep))

55452


In [95]:
# Combine the sample column from the transposed dataframe with all the columns from the "keep" list of headers
new_df=data_t[keep]

In [101]:
# Transpose and output the dataframe to csv so we have a copy of it
new_df.to_csv("trimmed.csv", index=False)